# Foot Stiffness Optimization to Minimize Metabolic Cost with Simulated Annealing

This notebook is to log each step performed in analyzing the results of our simulated annealing.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

with open('iteration_data.pkl', 'rb') as f:
    results = pickle.load(f)

print(results)

for result in results:
    print(result['iteration'])

[{'iteration': 2, 'x': array([16.64159604, 23.02357422,  6.33159604, 17.04386057,  6.33159604,
        6.33159604,  6.33159604,  6.33159604, 12.93386057, 17.04386057,
        9.73386057,  4.40386057, 18.92159604, 22.78386057,  9.33159604,
       15.06386056, 17.04386057,  6.33159604,  6.33159604, 15.06386056]), 'objective': 30.54244798398983, 'context': 0}, {'iteration': 3, 'x': array([ 7.9491203 , 14.33109848, 14.74560388,  8.35138483, 17.5391203 ,
       14.74560388, 17.5391203 , 14.74560388,  4.24138483,  5.55786841,
       20.94138482, 15.61138483,  7.43560388, 14.09138483, 20.5391203 ,
        3.5778684 ,  5.55786841, 14.74560388, 14.74560388,  3.5778684 ]), 'objective': 30.523363976940125, 'context': 0}, {'iteration': 7, 'x': array([16.29950126, 18.84252515,  7.1423586 ,  8.91643775, 18.87697178,
       20.31783909,  5.98950127, 20.02982912, 16.53813955,  6.8957199 ,
        4.84435654, 18.64155796, 12.71982913, 23.15022076,  3.45822533,
        6.39697343,  7.60417486,  6.369560